source: https://github.com/minoguep/rhyme_detection and https://paulminogue.com/index.php/2021/02/14/using-a-siamese-neural-network-to-create-a-simple-rhyme-detector/

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Subtract
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, CSVLogger
tqdm.pandas()

MAX_LEN = 64

2022-03-07 16:33:17.337084: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-07 16:33:17.337104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Step 1: Create dataset
We want equally many positive and negative samples of rhyme pairs

In [2]:
seed = 420

positive = pd.read_csv("../../norwegian_rhyme_scheme_corpus/annotation_tool/rhyme_pairs.tsv", sep="\t", names=["word_a", "word_b"], header=0)
negative = pd.read_csv("../../norwegian_rhyme_scheme_corpus/annotation_tool/negative_rhyme_pairs.tsv", sep="\t", names=["word_a", "word_b"], header=0)
negative = negative.sample(n=len(positive), random_state=seed)
positive["rhyme"] = [1]*len(positive)
negative["rhyme"] = [0]*len(negative)

df = pd.concat([positive, negative])

In [3]:
df

,word_a,word_b,rhyme
0,huse,bruse,1
1,halen,pralen,1
2,oksepar,svar,1
3,bevare,bare,1
4,ly,våbengny,1
...,...,...,...
1757,støy,brast,0
20575,meier,kne,0
2632,synger,land,0
16537,frukt,sten,0


### Step 2: Create model
Copy paste from Pauls notebook

In [4]:
def tokenize_inputs(phrase_a, phrase_b, tokenizer):
    tokenized_phrases = tokenizer.texts_to_sequences([phrase_a, phrase_b])

    # now loop through inputs and pad or reduce size if required
    tokenized_phrases_for_output = []
    for phrase in tokenized_phrases:
        if len(phrase) < MAX_LEN:
            length_to_pad = MAX_LEN - len(phrase)
            phrase_for_output = ([0] * length_to_pad) + phrase
        elif len(phrase) > MAX_LEN:
            phrase_for_output = phrase[-MAX_LEN:]
        else:
            phrase_for_output = phrase
        tokenized_phrases_for_output.append(phrase_for_output)

    return tf.constant(tokenized_phrases_for_output, dtype=tf.float64)

In [5]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(df['word_a'] + df['word_b'])

df['word_tokens'] = df.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

  0%|          | 0/14506 [00:00<?, ?it/s]

2022-03-07 16:33:18.575808: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-07 16:33:18.575836: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-07 16:33:18.575857: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tita-laptop): /proc/driver/nvidia/version does not exist
2022-03-07 16:33:18.576094: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def create_model():
  
    word_a_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_a_input_tokens'
      )
    word_b_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_b_input_tokens'
      )
    
    # This is the siamese portion of the model 
    common_lstm = LSTM(64, return_sequences=False, activation="relu", name="common_lstm_layer")

    word_a_lstm_output = common_lstm(word_a_input_tokens)
    word_b_lstm_output = common_lstm(word_b_input_tokens)

    #concatenate_lstm_outputs
    concat_layer = Subtract(name="concatenate_lstm_outputs")(
      [word_a_lstm_output, word_b_lstm_output]
      )
    
    # dense layers before final classification
    dense_layers = Dense(64, activation="relu", name="first_dense_layer")(concat_layer)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(32, activation="relu", name="second_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(8, activation="relu", name="third_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    classification_layer = Dense(1, activation="sigmoid", name="classification_layer")(dense_layers)
    
    model = Model(
      inputs=[word_a_input_tokens, word_b_input_tokens], 
      outputs = classification_layer
      )

    model.compile(
      loss="binary_crossentropy",
      metrics=["accuracy"],
      optimizer="Adam"
    )

    return model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    list(df['word_tokens']), list(df['rhyme']), stratify=df['rhyme'], 
    test_size=0.4, random_state=seed
    )
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.25, random_state=seed
    )

In [8]:
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_val = tf.convert_to_tensor(X_val)
y_val = tf.convert_to_tensor(y_val)

In [9]:
X_train.shape, X_val.shape, len(X_test)

(TensorShape([8703, 2, 64]), TensorShape([1451, 2, 64]), 4352)

In [10]:
print(f"""
    Data set size: Full set: {len(df)}
    Train: {len(X_train)}
    Dev: {len(X_val)}
    Test: {len(X_test)}
""")



    Data set size: Full set: 14506
    Train: 8703
    Dev: 1451
    Test: 4352



In [11]:
# #Uncomment to train model 

# model = create_model()

# model_checkpoint = ModelCheckpoint("models/rhyme_model.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger('training.log')

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )


In [12]:
# load the model
model = load_model("models/rhyme_model.hdf5")

X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      2176
           1       0.97      0.90      0.93      2176

    accuracy                           0.94      4352
   macro avg       0.94      0.94      0.94      4352
weighted avg       0.94      0.94      0.94      4352



In [13]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)
sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Sentence 1: {samples[i][0]}")
    print(f"Sentence 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

Sentence 1: Kan du ikke se det
Sentence 2: Deg skal jeg lede
Non-rhyme(0.2574000060558319)
---------------

Sentence 1: Kaker av alle slag
Sentence 2: Her henger Norges flagg
Rhyme(0.987500011920929)
---------------

Sentence 1: Jeg har ikke tid
Sentence 2: Til dette svineri
Non-rhyme(0.27000001072883606)
---------------

Sentence 1: Hva har du sagt
Sentence 2: Kaken er bakt
Rhyme(0.9990000128746033)
---------------

Sentence 1: Barna er lagt
Sentence 2: Kaken er laget
Non-rhyme(0.006500000134110451)
---------------

Sentence 1: Gjorde du det med vilje
Sentence 2: Kaken smaker vanilje
Rhyme(0.9991000294685364)
---------------

Sentence 1: Dette vokser
Sentence 2: Satans underbukser
Rhyme(0.9991000294685364)
---------------



# Try again with different 2:3 positive to negative ratio
Use same test and dev sets, and same tokenizer. Only expand training set.  
Train set is already 50/50 positive and negative. We want to make it 40/60 

In [35]:
new_neg = (len(X_train)//4)
new_neg

2175

In [33]:
seed = 420

negative = pd.read_csv("../../norwegian_rhyme_scheme_corpus/annotation_tool/negative_rhyme_pairs.tsv", sep="\t", names=["word_a", "word_b"], header=0)
# use same seed, and extract all the pairs already used + the ones we need 
negative2 = negative.sample(n=len(positive) + new_neg, random_state=seed)
negative2["rhyme"] = [0]*len(negative2)


In [36]:
unused_neg = negative2[len(positive):]
unused_neg

,word_a,word_b,rhyme
6396,fører,inn,0
12814,skrider,vadested,0
11958,trenger,lønn,0
1672,vende,snare,0
3198,rund,ild,0
...,...,...,...
7863,sprenger,fanger,0
21161,flom,lande,0
13101,krypet,fred,0
15024,fritt,finne,0


In [37]:
# use same tokenizer as above
unused_neg['word_tokens'] = unused_neg.progress_apply(
    lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1
)

X_train_new = tf.convert_to_tensor(list(X_train) + list(unused_neg["word_tokens"]))
y_train_new = tf.convert_to_tensor(list(y_train) + list(unused_neg["rhyme"]))


  0%|          | 0/2175 [00:00<?, ?it/s]

/tmp/ipykernel_171726/1257613138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unused_neg['word_tokens'] = unused_neg.progress_apply(


In [38]:
# Uncomment to train model
model = create_model()
                                    # changed model name
model_checkpoint = ModelCheckpoint("models/rhyme_model_2.hdf5",monitor="val_loss")
terminate_on_nan = TerminateOnNaN()
csv_logger = CSVLogger('training2.log')

history = model.fit(
    [X_train_new[:, 0], X_train_new[:, 1]],
    y_train_new,
    batch_size=128,
    epochs=100,
    callbacks=[model_checkpoint, terminate_on_nan, csv_logger],
    validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
)

Epoch 1/100
85/85 [==============================] - 5s 44ms/step - loss: 0.6259 - accuracy: 0.5902 - val_loss: 0.5785 - val_accuracy: 0.4997
Epoch 2/100
85/85 [==============================] - 4s 42ms/step - loss: 0.5336 - accuracy: 0.7488 - val_loss: 0.5388 - val_accuracy: 0.8560
Epoch 3/100
85/85 [==============================] - 4s 41ms/step - loss: 0.5019 - accuracy: 0.8578 - val_loss: 0.5108 - val_accuracy: 0.8725
Epoch 4/100
85/85 [==============================] - 4s 43ms/step - loss: 0.4803 - accuracy: 0.8689 - val_loss: 0.4959 - val_accuracy: 0.8773
Epoch 5/100
85/85 [==============================] - 4s 42ms/step - loss: 0.4590 - accuracy: 0.8758 - val_loss: 0.4695 - val_accuracy: 0.8897
Epoch 6/100
85/85 [==============================] - 4s 42ms/step - loss: 0.4341 - accuracy: 0.8798 - val_loss: 0.4630 - val_accuracy: 0.8835
Epoch 7/100
85/85 [==============================] - 3s 39ms/step - loss: 0.4120 - accuracy: 0.8880 - val_loss: 0.4352 - val_accuracy: 0.8904
Epoch 

85/85 [==============================] - 3s 40ms/step - loss: 0.2214 - accuracy: 0.9255 - val_loss: 0.2886 - val_accuracy: 0.9407
Epoch 59/100
85/85 [==============================] - 3s 41ms/step - loss: 0.2209 - accuracy: 0.9289 - val_loss: 0.2583 - val_accuracy: 0.9449
Epoch 60/100
85/85 [==============================] - 3s 41ms/step - loss: 0.2174 - accuracy: 0.9285 - val_loss: 0.2870 - val_accuracy: 0.9366
Epoch 61/100
85/85 [==============================] - 4s 46ms/step - loss: 0.2022 - accuracy: 0.9362 - val_loss: 0.2562 - val_accuracy: 0.9449
Epoch 62/100
85/85 [==============================] - 4s 47ms/step - loss: 0.2071 - accuracy: 0.9286 - val_loss: 0.2913 - val_accuracy: 0.9428
Epoch 63/100
85/85 [==============================] - 4s 44ms/step - loss: 0.2009 - accuracy: 0.9329 - val_loss: 0.2737 - val_accuracy: 0.9414
Epoch 64/100
85/85 [==============================] - 4s 46ms/step - loss: 0.2115 - accuracy: 0.9304 - val_loss: 0.2603 - val_accuracy: 0.9490
Epoch 65/100

In [39]:
# load the model
model = load_model("models/rhyme_model_2.hdf5")

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2176
           1       0.95      0.94      0.95      2176

    accuracy                           0.95      4352
   macro avg       0.95      0.95      0.95      4352
weighted avg       0.95      0.95      0.95      4352



In [41]:
samples = [
    ["Kan du ikke se det", "Deg skal jeg lede"], 
    ["Kaker av alle slag", "Her henger Norges flagg"], 
    ["Jeg har ikke tid", "Til dette svineri"],
    ["Hva har du sagt", "Kaken er bakt"], 
    ["Barna er lagt", "Kaken er laget"],
    ["Gjorde du det med vilje", "Kaken smaker vanilje"], 
    ["Dette vokser", "Satans underbukser"],
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)
sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Lyric 1: {samples[i][0]}")
    print(f"Lyric 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

Lyric 1: Kan du ikke se det
Lyric 2: Deg skal jeg lede
Non-rhyme(0.27219998836517334)
---------------

Lyric 1: Kaker av alle slag
Lyric 2: Her henger Norges flagg
Rhyme(0.6155999898910522)
---------------

Lyric 1: Jeg har ikke tid
Lyric 2: Til dette svineri
Non-rhyme(0.36890000104904175)
---------------

Lyric 1: Hva har du sagt
Lyric 2: Kaken er bakt
Rhyme(0.9211000204086304)
---------------

Lyric 1: Barna er lagt
Lyric 2: Kaken er laget
Non-rhyme(0.0)
---------------

Lyric 1: Gjorde du det med vilje
Lyric 2: Kaken smaker vanilje
Rhyme(0.9211000204086304)
---------------

Lyric 1: Dette vokser
Lyric 2: Satans underbukser
Rhyme(0.9211000204086304)
---------------



## Try to use whole sentences instead of just word pairs

In [21]:
# from collections import Counter, defaultdict
# import itertools


# seed = 420

# def equal_class_numbers(scheme):
#     c = Counter(scheme)
#     return len(set(c.values())) == 1

# df = pd.read_csv("../../norwegian_rhyme_scheme_corpus/annotation_tool/tsvs/tita_rhymes_poems.tsv", sep="\t")
# balanced_schemes_df = df.loc[df["rhyme scheme"].apply(equal_class_numbers)]
# balanced_schemes_df

# positive = set()
# negative = set()

# j = 0

# for e in balanced_schemes_df.itertuples():
#     lines = e.stanza.split("\n")
#     code = e._1
#     d = defaultdict(set)
#     if "I" in code or "N" in code or "T" in code:
#         continue
#     for c, line in zip(code, lines):
#         d[c].add(line)
        
#     # Positive examples = all combinations within same rhyme scheme letter
#     for s in d.values():
#         c = itertools.combinations(s, 2)
#         [positive.add(e) for e in c]
    
#     # Negative examples = all lines with one letter combined with all lines for all other letters
#     keys = list(d.keys())
#     for i, key in enumerate(keys):
#         for j in range(i+1, len(keys)):
#             key2 = keys[j]
#             for l1 in d[key]:
#                 for l2 in d[key2]:
#                     negative.add((l1, l2))
                    
# print(len(positive), len(negative))

# positive_df = pd.DataFrame(positive, columns=["sent_a", "sent_b"])
# negative_df = pd.DataFrame(negative, columns=["sent_a", "sent_b"])

# negative_df = negative_df.sample(n=len(positive_df), random_state=seed)
# positive_df["rhyme"] = [1]*len(positive_df)
# negative_df["rhyme"] = [0]*len(negative_df)

# df = pd.concat([positive_df, negative_df])
# df

In [22]:
# #copy paste from above cells
# tokenizer = Tokenizer(char_level=True, lower=True)
# tokenizer.fit_on_texts(df['sent_a'] + df['sent_b'])

# df['sent_tokens'] = df.progress_apply(
#     lambda row: tokenize_inputs(row['sent_a'], row['sent_b'], tokenizer), axis=1
# )

# X_train, X_test, y_train, y_test = train_test_split(
#     list(df['sent_tokens']), list(df['rhyme']), stratify=df['rhyme'], 
#     test_size=0.4, random_state=seed
#     )
# X_test, X_val, y_test, y_val = train_test_split(
#     X_test, y_test, stratify=y_test, test_size=0.25, random_state=seed
#     )

# X_train = tf.convert_to_tensor(X_train)
# y_train = tf.convert_to_tensor(y_train)
# X_val = tf.convert_to_tensor(X_val)
# y_val = tf.convert_to_tensor(y_val)

In [23]:
# model = create_model()
#                                     # changed model name
# model_checkpoint = ModelCheckpoint("models/rhyme_model_3.hdf5",monitor="val_loss")
# terminate_on_nan = TerminateOnNaN()
# csv_logger = CSVLogger('training3.log')

# history = model.fit(
#     [X_train[:, 0], X_train[:, 1]],
#     y_train,
#     batch_size=128,
#     epochs=100,
#     callbacks=[model_checkpoint, terminate_on_nan, csv_logger],
#     validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
# )

In [24]:
# # load the model
# model = load_model("models/rhyme_model_3.hdf5")

# X_test = tf.convert_to_tensor(X_test)
# y_test = tf.convert_to_tensor(y_test)

# y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
# y_pred = y_pred > 0.5
# print(classification_report(y_test, y_pred))

In [25]:
# samples = [
#     ["Kan du ikke se det?", "Deg skal jeg lede"], 
#     ["Kaker av alle slag", "Her henger Norges flagg"], 
#     ["Jeg har ikke tid", "Til dette svineri"],
#     ["Hva har du sagt", "Kaken er bakt"], 
#     ["Barna er lagt", "Kaken er laget"],
#     ["Er du sikker på at dette er med vilje?", "Hjertet ditt smaker vanilje"], 
#     ["Dette vokser", "Satans underbukser"],
# ]

# sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
# sample_tokens = tf.convert_to_tensor(sample_tokens)
# sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
# predictions = [round(pred[0], 4) for pred in sample_pred]
# for i in range(len(samples)):
#     print(f"Lyric 1: {samples[i][0]}")
#     print(f"Lyric 2: {samples[i][1]}")
#     print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
#     print("---------------\n")